code from https://www.pyimagesearch.com/2018/05/07/multi-label-classification-with-keras/

구글 드라이브가 연결되어 있고, 구글 드라이브의 tmp 폴더 안에 fashion_dataset.zip, fashion_examples.zip이 있을 때,

In [21]:
!ls -al "/content/drive/My Drive/tmp/"

total 4140519
drwx------ 2 root root      4096 Jun 12  2019  부산카톨릭대_교재
-rw------- 1 root root 210504772 Apr 25 00:42  1000명_croped_images.zip
-rw------- 1 root root    957336 Dec 27 04:43  나혼자만레벨업+1-243외전21,후일담6_001.txt
-rw------- 1 root root    957336 Dec 27 04:43  나혼자만레벨업+1-243외전21,후일담6_002.txt
-rw------- 1 root root    957335 Dec 27 04:43  나혼자만레벨업+1-243외전21,후일담6_003.txt
drwx------ 2 root root      4096 Jun 18  2019  201906_중앙학원_과제신청
-rw------- 1 root root      3478 May  8 06:23  20200505_새로추가와_비적절제거_label.zip
-rw------- 1 root root 300574287 May  6 22:49  20200505_새로추가와_비적절제거.zip
-rw------- 1 root root     33910 Sep 19  2019  6차.ipynb
-rw------- 1 root root     16007 Sep 19  2019  7회차.ipynb
-rw------- 1 root root       151 May  8  2019  AWS_인터뷰_응답메일.gdoc
-rw------- 1 root root    301774 May  8  2019  AWS_인터뷰_응답메일.pdf
-rw------- 1 root root 188909928 Ap

In [0]:
!ls -al "/content/drive/My Drive/tmp/fashion_dataset.zip"

-rw------- 1 root root 537686706 May 29 05:41 '/content/drive/My Drive/tmp/fashion_dataset.zip'


In [0]:
!cp "/content/drive/My Drive/tmp/fashion_dataset.zip" ./
!cp "/content/drive/My Drive/tmp/fashion_examples.zip" ./

In [23]:
!ls -al

total 525328
drwxr-xr-x 1 root root      4096 May 29 06:29 .
drwxr-xr-x 1 root root      4096 May 29 05:35 ..
drwxr-xr-x 1 root root      4096 May 27 16:27 .config
drwx------ 4 root root      4096 May 29 06:09 drive
-rw-r--r-- 1 root root 537686706 May 29 06:34 fashion_dataset.zip
-rw------- 1 root root    215899 May 29 06:34 fashion_examples.zip
drwxr-xr-x 2 root root      4096 May 29 06:09 .ipynb_checkpoints
drwxr-xr-x 1 root root      4096 May 27 16:27 sample_data


In [0]:
!unzip fashion_dataset.zip

Archive:  fashion_dataset.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of fashion_dataset.zip or
        fashion_dataset.zip.zip, and cannot find fashion_dataset.zip.ZIP, period.


In [0]:
!unzip 

In [0]:
!ls -al


total 261140
drwxr-xr-x 1 root root      4096 May 29 05:42 .
drwxr-xr-x 1 root root      4096 May 29 05:35 ..
drwxr-xr-x 1 root root      4096 May 27 16:27 .config
-rw-r--r-- 1 root root 267386880 May 29 06:07 fashion_dataset.zip
drwxr-xr-x 1 root root      4096 May 27 16:27 sample_data


In [0]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
# matplotlib.use("Agg")

# import the necessary packages
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
# from pyimagesearch.fashionnet import FashionNet
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2
import os

In [0]:
# USAGE
# python train.py --dataset dataset --model output/fashion.model \
#	--categorybin output/category_lb.pickle --colorbin output/color_lb.pickle

# # construct the argument parse and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-d", "--dataset", required=True,
# 	help="path to input dataset (i.e., directory of images)")
# ap.add_argument("-m", "--model", required=True,
# 	help="path to output model")
# ap.add_argument("-l", "--categorybin", required=True,
# 	help="path to output category label binarizer")
# ap.add_argument("-c", "--colorbin", required=True,
# 	help="path to output color label binarizer")
# ap.add_argument("-p", "--plot", type=str, default="output",
# 	help="base filename for generated plots")
# args = vars(ap.parse_args())

args = { "dataset":"multi-output-classification/dataset", 
        "model":"multi-output-classification/output/fashion.model", 
        "categorybin":"multi-output-classification/output/category_lb.pickle", 
        "colorbin":"multi-output-classification/output/color_lb.pickle", 
        "plot":"multi-output-classification/output" }


# initialize the number of epochs to train for, initial learning rate,
# batch size, and image dimensions
EPOCHS = 50
INIT_LR = 1e-3
BS = 32
IMAGE_DIMS = (96, 96, 3)


# grab the image paths and randomly shuffle them
print("[INFO] loading images...")
imagePaths = sorted(list(paths.list_images(args["dataset"])))
random.seed(42)
random.shuffle(imagePaths)

# initialize the data, clothing category labels (i.e., shirts, jeans,
# dresses, etc.) along with the color labels (i.e., red, blue, etc.)
data = []
categoryLabels = []
colorLabels = []

# loop over the input images
for imagePath in imagePaths:
	# load the image, pre-process it, and store it in the data list
	image = cv2.imread(imagePath)
	image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	image = img_to_array(image)
	data.append(image)

	# extract the clothing color and category from the path and
	# update the respective lists
	(color, cat) = imagePath.split(os.path.sep)[-2].split("_")
	categoryLabels.append(cat)
	colorLabels.append(color)

# scale the raw pixel intensities to the range [0, 1] and convert to
# a NumPy array
data = np.array(data, dtype="float") / 255.0
print("[INFO] data matrix: {} images ({:.2f}MB)".format(
	len(imagePaths), data.nbytes / (1024 * 1000.0)))

# convert the label lists to NumPy arrays prior to binarization
categoryLabels = np.array(categoryLabels)
colorLabels = np.array(colorLabels)

# binarize both sets of labels
print("[INFO] binarizing labels...")
categoryLB = LabelBinarizer()
colorLB = LabelBinarizer()
categoryLabels = categoryLB.fit_transform(categoryLabels)
colorLabels = colorLB.fit_transform(colorLabels)

# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
split = train_test_split(data, categoryLabels, colorLabels, test_size=0.2, random_state=42)
(trainX, testX, trainCategoryY, testCategoryY, trainColorY, testColorY) = split


[INFO] loading images...
[INFO] data matrix: 2521 images (544.54MB)
[INFO] binarizing labels...


In [0]:
# import the necessary packages
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Dropout
from keras.layers.core import Lambda
from keras.layers.core import Dense
from keras.layers import Flatten
from keras.layers import Input
import tensorflow as tf

class FashionNet:
	@staticmethod
	def build_category_branch(inputs, numCategories,
		finalAct="softmax", chanDim=-1):
		# utilize a lambda layer to convert the 3 channel input to a
		# grayscale representation
		x = Lambda(lambda c: tf.image.rgb_to_grayscale(c))(inputs)

		# CONV => RELU => POOL
		x = Conv2D(32, (3, 3), padding="same")(x)
		x = Activation("relu")(x)
		x = BatchNormalization(axis=chanDim)(x)
		x = MaxPooling2D(pool_size=(3, 3))(x)
		x = Dropout(0.25)(x)

		# (CONV => RELU) * 2 => POOL
		x = Conv2D(64, (3, 3), padding="same")(x)
		x = Activation("relu")(x)
		x = BatchNormalization(axis=chanDim)(x)
		x = Conv2D(64, (3, 3), padding="same")(x)
		x = Activation("relu")(x)
		x = BatchNormalization(axis=chanDim)(x)
		x = MaxPooling2D(pool_size=(2, 2))(x)
		x = Dropout(0.25)(x)

		# (CONV => RELU) * 2 => POOL
		x = Conv2D(128, (3, 3), padding="same")(x)
		x = Activation("relu")(x)
		x = BatchNormalization(axis=chanDim)(x)
		x = Conv2D(128, (3, 3), padding="same")(x)
		x = Activation("relu")(x)
		x = BatchNormalization(axis=chanDim)(x)
		x = MaxPooling2D(pool_size=(2, 2))(x)
		x = Dropout(0.25)(x)

		# define a branch of output layers for the number of different
		# clothing categories (i.e., shirts, jeans, dresses, etc.)
		x = Flatten()(x)
		x = Dense(256)(x)
		x = Activation("relu")(x)
		x = BatchNormalization()(x)
		x = Dropout(0.5)(x)
		x = Dense(numCategories)(x)
		x = Activation(finalAct, name="category_output")(x)

		# return the category prediction sub-network
		return x

	@staticmethod
	def build_color_branch(inputs, numColors, finalAct="softmax",
		chanDim=-1):
		# CONV => RELU => POOL
		x = Conv2D(16, (3, 3), padding="same")(inputs)
		x = Activation("relu")(x)
		x = BatchNormalization(axis=chanDim)(x)
		x = MaxPooling2D(pool_size=(3, 3))(x)
		x = Dropout(0.25)(x)

		# CONV => RELU => POOL
		x = Conv2D(32, (3, 3), padding="same")(x)
		x = Activation("relu")(x)
		x = BatchNormalization(axis=chanDim)(x)
		x = MaxPooling2D(pool_size=(2, 2))(x)
		x = Dropout(0.25)(x)

		# CONV => RELU => POOL
		x = Conv2D(32, (3, 3), padding="same")(x)
		x = Activation("relu")(x)
		x = BatchNormalization(axis=chanDim)(x)
		x = MaxPooling2D(pool_size=(2, 2))(x)
		x = Dropout(0.25)(x)

		# define a branch of output layers for the number of different
		# colors (i.e., red, black, blue, etc.)
		x = Flatten()(x)
		x = Dense(128)(x)
		x = Activation("relu")(x)
		x = BatchNormalization()(x)
		x = Dropout(0.5)(x)
		x = Dense(numColors)(x)
		x = Activation(finalAct, name="color_output")(x)

		# return the color prediction sub-network
		return x

	@staticmethod
	def build(width, height, numCategories, numColors,
		finalAct="softmax"):
		# initialize the input shape and channel dimension (this code
		# assumes you are using TensorFlow which utilizes channels
		# last ordering)
		inputShape = (height, width, 3)
		chanDim = -1

		# construct both the "category" and "color" sub-networks
		inputs = Input(shape=inputShape)
		categoryBranch = FashionNet.build_category_branch(inputs,
			numCategories, finalAct=finalAct, chanDim=chanDim)
		colorBranch = FashionNet.build_color_branch(inputs,
			numColors, finalAct=finalAct, chanDim=chanDim)

		# create the model using our input (the batch of images) and
		# two separate outputs -- one for the clothing category
		# branch and another for the color branch, respectively
		model = Model(
			inputs=inputs,
			outputs=[categoryBranch, colorBranch],
			name="fashionnet")

		# return the constructed network architecture
		return model

In [0]:
# initialize our FashionNet multi-output network
model = FashionNet.build(96, 96,
	numCategories=len(categoryLB.classes_),
	numColors=len(colorLB.classes_),
	finalAct="softmax")

# define two dictionaries: one that specifies the loss method for
# each output of the network along with a second dictionary that
# specifies the weight per loss
losses = {
	"category_output": "categorical_crossentropy",
	"color_output": "categorical_crossentropy",
}
lossWeights = {"category_output": 1.0, "color_output": 1.0}

# initialize the optimizer and compile the model
print("[INFO] compiling model...")
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(optimizer=opt, loss=losses, loss_weights=lossWeights,
	metrics=["accuracy"])

# train the network to perform multi-output classification
H = model.fit(trainX,
	{"category_output": trainCategoryY, "color_output": trainColorY},
	validation_data=(testX,
		{"category_output": testCategoryY, "color_output": testColorY}),
	epochs=EPOCHS,
	verbose=1)

# save the model to disk
print("[INFO] serializing network...")
model.save(args["model"])

# save the category binarizer to disk
print("[INFO] serializing category label binarizer...")
f = open(args["categorybin"], "wb")
f.write(pickle.dumps(categoryLB))
f.close()

# save the color binarizer to disk
print("[INFO] serializing color label binarizer...")
f = open(args["colorbin"], "wb")
f.write(pickle.dumps(colorLB))
f.close()

[INFO] compiling model...
Train on 2016 samples, validate on 505 samples
Epoch 1/50
2016/2016 [==============================] - 10s 5ms/step - loss: 0.9572 - category_output_loss: 0.6167 - color_output_loss: 0.3404 - category_output_accuracy: 0.8175 - color_output_accuracy: 0.8735 - val_loss: 2.8219 - val_category_output_loss: 1.6293 - val_color_output_loss: 1.1912 - val_category_output_accuracy: 0.3188 - val_color_output_accuracy: 0.4475
Epoch 2/50
2016/2016 [==============================] - 7s 3ms/step - loss: 0.3694 - category_output_loss: 0.2550 - color_output_loss: 0.1144 - category_output_accuracy: 0.9201 - color_output_accuracy: 0.9588 - val_loss: 2.9823 - val_category_output_loss: 1.7012 - val_color_output_loss: 1.2766 - val_category_output_accuracy: 0.3644 - val_color_output_accuracy: 0.4475
Epoch 3/50
2016/2016 [==============================] - 7s 3ms/step - loss: 0.2580 - category_output_loss: 0.1821 - color_output_loss: 0.0759 - category_output_accuracy: 0.9410 - color_o

In [0]:
# plot the total loss, category loss, and color loss
lossNames = ["loss", "category_output_loss", "color_output_loss"]
plt.style.use("ggplot")
(fig, ax) = plt.subplots(3, 1, figsize=(13, 13))

# loop over the loss names
for (i, l) in enumerate(lossNames):
	# plot the loss for both the training and validation data
	title = "Loss for {}".format(l) if l != "loss" else "Total loss"
	ax[i].set_title(title)
	ax[i].set_xlabel("Epoch #")
	ax[i].set_ylabel("Loss")
	ax[i].plot(np.arange(0, EPOCHS), H.history[l], label=l)
	ax[i].plot(np.arange(0, EPOCHS), H.history["val_" + l],
		label="val_" + l)
	ax[i].legend()

# save the losses figure
plt.tight_layout()
plt.show()
# plt.savefig("{}_losses.png".format(args["plot"]))
plt.close()

# create a new figure for the accuracies
accuracyNames = ["category_output_accuracy", "color_output_accuracy"]
plt.style.use("ggplot")
(fig, ax) = plt.subplots(2, 1, figsize=(8, 8))

# loop over the accuracy names
for (i, l) in enumerate(accuracyNames):
	# plot the loss for both the training and validation data
	ax[i].set_title("Accuracy for {}".format(l))
	ax[i].set_xlabel("Epoch #")
	ax[i].set_ylabel("Accuracy")
	ax[i].plot(np.arange(0, EPOCHS), H.history[l], label=l)
	ax[i].plot(np.arange(0, EPOCHS), H.history["val_" + l],
		label="val_" + l)
	ax[i].legend()

# save the accuracies figure
plt.tight_layout()
plt.show()
# plt.savefig("{}_accs.png".format(args["plot"]))
plt.close()

In [0]:
# import the necessary packages
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import tensorflow as tf
import numpy as np
import argparse
import imutils
import pickle
import cv2

In [0]:
# USAGE
# python classify.py --model output/fashion.model \
#	--categorybin output/category_lb.pickle --colorbin output/color_lb.pickle \
#	--image examples/black_dress.jpg



# construct the argument parse and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-m", "--model", required=True,
# 	help="path to trained model model")
# ap.add_argument("-l", "--categorybin", required=True,
# 	help="path to output category label binarizer")
# ap.add_argument("-c", "--colorbin", required=True,
# 	help="path to output color label binarizer")
# ap.add_argument("-i", "--image", required=True,
# 	help="path to input image")
# args = vars(ap.parse_args())



args = { "model":"multi-output-classification/output/fashion.model", 
        "categorybin":"multi-output-classification/output/category_lb.pickle", 
        "colorbin":"multi-output-classification/output/color_lb.pickle", 
        "image":"multi-output-classification/examples/black_dress.jpg" }

# load the image
image = cv2.imread(args["image"])
output = imutils.resize(image, width=400)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# pre-process the image for classification
image = cv2.resize(image, (96, 96))
image = image.astype("float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)

# load the trained convolutional neural network from disk, followed
# by the category and color label binarizers, respectively
print("[INFO] loading network...")
model = load_model(args["model"], custom_objects={"tf": tf})
categoryLB = pickle.loads(open(args["categorybin"], "rb").read())
colorLB = pickle.loads(open(args["colorbin"], "rb").read())

# classify the input image using Keras' multi-output functionality
print("[INFO] classifying image...")
(categoryProba, colorProba) = model.predict(image)

plt.imshow(image.reshape(IMAGE_DIMS))
plt.show()
categoryIdx = categoryProba[0].argmax()
colorIdx = colorProba[0].argmax()
categoryLabel = categoryLB.classes_[categoryIdx]
colorLabel = colorLB.classes_[colorIdx]

# draw the category label and color label on the image
categoryText = "category: {} ({:.2f}%)".format(categoryLabel,
	categoryProba[0][categoryIdx] * 100)
colorText = "color: {} ({:.2f}%)".format(colorLabel,
	colorProba[0][colorIdx] * 100)
print(categoryText)
print(colorText)


[INFO] loading network...
[INFO] classifying image...
category: shirt (93.84%)
color: black (99.11%)
